In [1]:
import numpy as np # import for math and array operations
import pandas as pd # import for dataframe handle
import matplotlib.pyplot as plt # import for visual representation
import seaborn as sns # import for visual representation
import math
from bs4 import BeautifulSoup
import requests
import datetime as dt

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('../raw_data/combine_df.csv')

In [4]:
def clean_tags(tag):
    if ',' in str(tag):
        tag = ','.join(list(set(str(tag).split(','))))
        return tag
df['tags'] = df["popular_tags"].astype(str) +',' + df['genre'].astype(str)
df['tags'] = df['tags'].apply(lambda x : clean_tags(x))

In [6]:
df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,...,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,clean_reviews,clean_date,tags
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...",...,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99,Very Positive,2016-05-12 00:00:00,"Co-op,Zombies,Shooter,Remake,Atmospheric,Memes..."
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...",...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN,Mixed,2017-12-21 00:00:00,"Team-Based,Co-op,Competitive,Adventure,Shooter..."
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN,Mostly Positive,2018-04-24 00:00:00,"Turn-Based Strategy,Adventure,Turn-Based Tacti..."
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...",...,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN,Mixed,2018-12-13 00:00:00,"Co-op,Zombies,Adventure,Shooter,Post-apocalypt..."
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...",...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN,Mostly Positive,2003-05-06 00:00:00,"Economy,Atmospheric,Space,Massively Multiplaye..."


In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from unidecode import unidecode
import string

def clean (text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('English')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    return " ".join(without_stopwords)

In [8]:
df['clean_tag'] = df['tags'].apply(clean)

In [14]:
df = df.drop(['tags','genre' , 'popular_tags'],axis  =1)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tuned TFidfvectorizer
vec = TfidfVectorizer(min_df = 0.01).fit(df.clean_tag)
vectors = vec.transform(df.clean_tag) # Transform text to vectors


In [35]:
X_proj = pd.DataFrame(vectors.toarray(), index=df.name.tolist())

from sklearn.neighbors import KNeighborsRegressor

# Define X and y
y = X_proj[0]
knn_model = KNeighborsRegressor().fit(X_proj,y) 

In [42]:
#Test

neighbors_list = list(knn_model.kneighbors(X_proj.iloc[3500:3501],n_neighbors=10)[1][0][1:])
X_proj.iloc[neighbors_list, :][[0]]

,0
Tsundere Idol,0.0
Loca-Love My Cute Roommate,0.0
Momoiro Closet,0.0
Corona Blossom Vol.3 Journey to the Stars,0.0
Corona Blossom Vol.2 The Truth From Beyond,0.0
Corona Blossom Vol.1 Gift From the Galaxy,0.0
The Afterglow of Grisaia,0.0
The Leisure of Grisaia,0.0
NEKO-NIN exHeart 2 Love +PLUS,0.0
